In [4]:
import tkinter as tk
from tkinter import messagebox, simpledialog
from tkinter import ttk
from datetime import datetime, date
import json
import os

class ToDoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("To-Do List")
        self.root.geometry("600x550")
        self.root.configure(bg="white")

        self.tasks = []
        self.filename = "tasks.json"

        self.create_widgets()
        self.load_tasks()
        self.refresh_task_list()

    def create_widgets(self):
        title = tk.Label(self.root, text="To-Do List", font=("Helvetica", 24, "bold"), bg="white", fg="#4a90e2")
        title.pack(pady=10)

        form_frame = tk.Frame(self.root, bg="white")
        form_frame.pack(pady=5)

        tk.Label(form_frame, text="Task:", bg="white").grid(row=0, column=0, sticky="w")
        self.task_entry = tk.Entry(form_frame, width=40)
        self.task_entry.grid(row=0, column=1, padx=5)

        tk.Label(form_frame, text="Due Date (DD/MM/YYYY):", bg="white").grid(row=1, column=0, sticky="w")
        self.due_entry = tk.Entry(form_frame, width=20)
        self.due_entry.grid(row=1, column=1, sticky="w")

        add_btn = tk.Button(form_frame, text="Add Task", command=self.add_task, bg="#4caf50", fg="white")
        add_btn.grid(row=2, column=1, sticky="e", pady=5)

        filter_frame = tk.Frame(self.root, bg="white")
        filter_frame.pack(pady=5)

        self.filter_var = tk.StringVar(value="All")
        options = ["All", "Today", "Overdue", "Completed", "Incomplete"]
        self.filter_menu = ttk.Combobox(filter_frame, textvariable=self.filter_var, values=options, state="readonly")
        self.filter_menu.pack()
        self.filter_menu.bind("<<ComboboxSelected>>", lambda e: self.refresh_task_list())

        self.task_listbox = tk.Listbox(self.root, font=("Helvetica", 12), selectmode=tk.SINGLE, height=15)
        self.task_listbox.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

        button_frame = tk.Frame(self.root, bg="white")
        button_frame.pack(pady=5)

        tk.Button(button_frame, text="Mark Complete", command=self.mark_complete, bg="#2196f3", fg="white").grid(row=0, column=0, padx=5)
        tk.Button(button_frame, text="Edit", command=self.edit_task, bg="#ff9800", fg="white").grid(row=0, column=1, padx=5)
        tk.Button(button_frame, text="Delete", command=self.delete_task, bg="#f44336", fg="white").grid(row=0, column=2, padx=5)

        self.progress_var = tk.DoubleVar()
        self.progress = ttk.Progressbar(self.root, variable=self.progress_var, maximum=100)
        self.progress.pack(fill=tk.X, padx=20, pady=10)

    def add_task(self):
        task = self.task_entry.get().strip()
        due = self.due_entry.get().strip()

        if not task or not due:
            messagebox.showwarning("Input Error", "Please enter task and due date.")
            return

        try:
            datetime.strptime(due, "%d/%m/%Y")
        except ValueError:
            messagebox.showerror("Invalid Date", "Date format should be DD/MM/YYYY.")
            return

        self.tasks.append({"task": task, "due": due, "completed": False})
        self.save_tasks()
        self.task_entry.delete(0, tk.END)
        self.due_entry.delete(0, tk.END)
        self.refresh_task_list()

    def edit_task(self):
        idx = self.task_listbox.curselection()
        if not idx:
            return

        index = idx[0]
        task = self.filtered_tasks[index]
        new_task = simpledialog.askstring("Edit Task", "Edit task description:", initialvalue=task["task"])
        new_due = simpledialog.askstring("Edit Due Date", "Edit due date (DD/MM/YYYY):", initialvalue=task["due"])

        if new_task and new_due:
            try:
                datetime.strptime(new_due, "%d/%m/%Y")
                task["task"] = new_task
                task["due"] = new_due
                self.save_tasks()
                self.refresh_task_list()
            except ValueError:
                messagebox.showerror("Invalid Date", "Please enter a valid date (DD/MM/YYYY).")

    def delete_task(self):
        idx = self.task_listbox.curselection()
        if not idx:
            return
        index = idx[0]
        task = self.filtered_tasks[index]
        self.tasks.remove(task)
        self.save_tasks()
        self.refresh_task_list()

    def mark_complete(self):
        idx = self.task_listbox.curselection()
        if not idx:
            return
        index = idx[0]
        task = self.filtered_tasks[index]
        task["completed"] = True
        self.save_tasks()
        self.refresh_task_list()

    def refresh_task_list(self):
        self.task_listbox.delete(0, tk.END)
        selected_filter = self.filter_var.get()
        self.filtered_tasks = []
        completed = 0

        for task in self.tasks:
            try:
                due = datetime.strptime(task["due"], "%d/%m/%Y").date()
            except Exception:
                continue

            today = date.today()

            if selected_filter == "Today" and due != today:
                continue
            elif selected_filter == "Overdue" and (due >= today or task["completed"]):
                continue
            elif selected_filter == "Completed" and not task["completed"]:
                continue
            elif selected_filter == "Incomplete" and task["completed"]:
                continue

            status = "✔️" if task["completed"] else "❌"
            self.task_listbox.insert(tk.END, f"{status} {task['task']} (Due: {task['due']})")
            self.filtered_tasks.append(task)

            if task["completed"]:
                completed += 1

        total = len(self.tasks)
        self.progress_var.set((completed / total) * 100 if total > 0 else 0)

    def load_tasks(self):
        if os.path.exists(self.filename):
            with open(self.filename, "r") as f:
                self.tasks = json.load(f)

    def save_tasks(self):
        with open(self.filename, "w") as f:
            json.dump(self.tasks, f, indent=4)

if __name__ == "__main__":
    root = tk.Tk()
    app = ToDoApp(root)
    root.mainloop()
